In [34]:
import pandas as pd
from pymongo import MongoClient, GEO2D
import pymongo
import json
import datetime
import folium 
import math

In [35]:
#para saber el punto inicial y final de la linea
line_438_busstops = pd.read_csv('/home/kathrin/Doctorado/bus_rio/Datos_Omnibus/Onibus/Paradas/gtfs_linha438-paradas.csv')
departure_points = line_438_busstops[(line_438_busstops["sequencia"]==1)]
departure_points

,linha,descricao,agencia,sequencia,latitude,longitude
94,438,438-VILA ISABEL X LEBLON (VIA JOQUEI) - CIRCULAR,Fetranspor,1,-22.9882,-43.2283
127,438,438-VILA ISABEL X LEBLON (VIA JOQUEI) - CIRCULAR,Fetranspor,1,-22.9163,-43.2512


In [36]:
#Estableciendo la conexion con la base de datos aniv_cidade_event en MongoDB
db = MongoClient().aniv_cidade_event

In [37]:
#Funcion para determinar el punto inicial y final de un recorrido de una linea de omnibus
def bus_line_initial_final_points(bus_line):
    busstops_firstpoints=[]
    for row in db.busstops.find({ "linha": bus_line,"sequencia": 1 }):
        busstops_firstpoints.insert(0, [row[u'linha'], row[u'descricao'], row[u'latitude'], row[u'longitude']])

    #convertir la lista en un dataframe de pandas
    df_departure_points = pd.DataFrame(busstops_firstpoints, columns=['linha','descricao', 'latitude', 'longitude'])
    #df_departure_points
    initial_point = df_departure_points['latitude'][0], df_departure_points['longitude'][0]
    inverted_initial_point = df_departure_points['longitude'][0], df_departure_points['latitude'][0]
    final_point = df_departure_points['latitude'][1], df_departure_points['longitude'][1]
    inverted_final_point = df_departure_points['longitude'][1], df_departure_points['latitude'][1]
    return initial_point, final_point, inverted_initial_point, inverted_final_point

In [38]:
#Llamada a la funcion bus_line_initial_final_points de la 438
initial_final_points = bus_line_initial_final_points(438)
initial_final_points

((-22.9163, -43.251199999999997),
 (-22.988199999999999, -43.228299999999997),
 (-43.251199999999997, -22.9163),
 (-43.228299999999997, -22.988199999999999))

In [39]:
#De la misma llamada a la funcion de arriba retornar el valor en la posicion 1
initial_point = initial_final_points[1]
initial_point

(-22.988199999999999, -43.228299999999997)

In [40]:
#De la misma llamada a la funcion de arriba retornar el valor en la posicion 0
final_point = initial_final_points[0]
final_point

(-22.9163, -43.251199999999997)

In [41]:
inverted_initial_point = initial_final_points[3]
inverted_initial_point

(-43.228299999999997, -22.988199999999999)

In [42]:
inverted_final_point = initial_final_points[2]
inverted_final_point

(-43.251199999999997, -22.9163)

In [43]:
#Function to compute the bus_ids circulating during a day 20150215 of bus line 438
def bus_id_list ():
    bus_id_20150215_438 = db.dados20150215_line_438.distinct("bus_id")
    return bus_id_20150215_438
    

In [44]:
#llamada a la funcion bus_id_list
busId_list = bus_id_list()
busId_list

[u'A27556',
 u'A27610',
 u'A27617',
 u'A27604',
 u'A27510',
 u'A27603',
 u'A27599',
 u'A27514',
 u'A27534',
 u'A27632',
 u'A27629',
 u'A27550',
 u'A27558',
 u'A27521',
 u'A27606',
 u'A27565',
 u'A27504',
 u'A27633',
 u'A27566',
 u'A27554',
 u'A27600']

In [45]:
#Funcion para meter los bus_Id del dataframe para un array
def get_busId_list():
    array_busId = []
    for bus in busId_list:
        strbus = str(bus)
        array_busId.insert(len(array_busId), strbus)
    return array_busId  

In [46]:
array_busId = get_busId_list()
array_busId

['A27556',
 'A27610',
 'A27617',
 'A27604',
 'A27510',
 'A27603',
 'A27599',
 'A27514',
 'A27534',
 'A27632',
 'A27629',
 'A27550',
 'A27558',
 'A27521',
 'A27606',
 'A27565',
 'A27504',
 'A27633',
 'A27566',
 'A27554',
 'A27600']

In [47]:
len(array_busId)

21

In [48]:
#funcion que dado un el bus_id de un omnibus y un punto inicial/final devuelve los puntos mas cercanos a ese punto. 
# A esa función se le debería pasar también la BD, pero ahora no se va a hacer asi
def near_points_to_origin (bus_id, inverted_origin_point):
    near_initial_points_array = []
    #la funcion $within recibe las coordenadas en orden inverso (primero longitud y después latitud)
    for row in db.dados20150215_line_438.find({"latlon": {"$within": {"$center": [inverted_origin_point, 0.00052543]}}, "bus_id": bus_id}).sort("timestamp",pymongo.DESCENDING):
        near_initial_points_array.insert(0, [row[u'timestamp'], row[u'latitude'], row[u'longitude'], row[u'speed']])
    #convert the array in panda dataframe 
    sorted_near_initial_points = pd.DataFrame(near_initial_points_array, columns=['timestamp', 'latitude', 'longitude', 'speed'])
    return sorted_near_initial_points

In [49]:
#Funcion que retorna el arreglo con las horas de salida del omnibus al punto inicial/final del trayecto
def bus_departure_hour (initial_points_timestamp_list):
    list_departure_hours = []
    i = 0
    if len(initial_points_timestamp_list) >1:
        for i in range(0, len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            #si la diferencia entre los timestamp del omnibus en el putno supera los 40 minutos es q dio un viaje y regreso, sino lo supera es que se mantuvo ese tiempo esperando en el punto
            if (resto_total > 40):
                 list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S"))
        #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de  list_departure_hours 
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    else:
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    return list_departure_hours

In [50]:
#Funcion que retorna el arreglo con las horas de llegada del omnibus al punto inicial/final del trayecto
def bus_arrive_hour (initial_points_timestamp_list):
    list_arrive_hours = []
    #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de leblon_departure_hours 
    list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    if len(initial_points_timestamp_list) >1:
        for i in range(0,  len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            if (resto_total > 40):
                list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    return list_arrive_hours

In [51]:
#Para convertir de formato timedelta para horas y minutos
def days_hours_minutes(travel_time):
    hours = travel_time.seconds//3600 
    minutes = (travel_time.seconds//60)%60
    #print '%d hours, %d minutes' % (hours,minutes)
    if minutes < 10:
        minutes = str(0)+str(minutes)
    return str(hours) + ":"+ str(minutes)

In [52]:
#Funcion que crea la tabla de viajes con los tiempos de viaje
def create_travel_table (name_A, arrive_array_A, departure_array_A, name_B, arrive_array_B, departure_array_B, bus_Id, bus_line):
    
    my_time_table = []   
    num_travel = 0    
    i = 0 #Cursor para moverme por el departure_array_A(arreglo q contiene las horas de salida del punto incial)   
    j = 0 #Cursor para moverme por el departure_array_B (arreglo q contiene las horas de salida del punto final)
   
    while (i<len(departure_array_A) and j< len(departure_array_B)):
        if (departure_array_A[i] < departure_array_B[j]):
            direction = name_A + "_to_" + name_B
            departure_hour = departure_array_A[i]
            i = i +1
            arrive_hour = arrive_array_B[j]
            num_travel = num_travel +1
            travel_time = days_hours_minutes(arrive_hour - departure_hour)          
            my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
                        
        elif(departure_array_B[j] < departure_array_A[i]):
            direction = name_B + "_to_" + name_A
            departure_hour = departure_array_B[j]
            j = j +1
            arrive_hour = arrive_array_A[i]
            num_travel = num_travel +1
            travel_time = days_hours_minutes(arrive_hour - departure_hour)          
            my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            
    #Construir el dataframe pasandole el array con todos los datos de los viajes   
    mydf = pd.DataFrame(my_time_table, columns=['Line', 'Bus_Id','Direction','Departure_Hour', 'Arrive_Hour', 'Travel_Time'])       
    return mydf  


In [53]:
#Funcion que crea la tabla de tiempo de viaje para cada bus_id de los que transitaron el el dia
def all_travel_table (line, punto_ini_coord_invertidas, name_punto_inicio, punto_fin_coord_invertidas, name_punto_final, array_busId):
    new_df = pd.DataFrame(columns=['Line', 'Bus_Id','Direction','Departure_Hour', 'Arrive_Hour', 'Travel_Time'])  
    for busId in array_busId:
        sorted_near_initial_points1 = near_points_to_origin(busId, punto_ini_coord_invertidas)
        sorted_near_final_points1 = near_points_to_origin(busId, punto_fin_coord_invertidas)
        initial_points_timestamp1 = sorted_near_initial_points1 ['timestamp']
        final_points_timestamp1 = sorted_near_final_points1 ['timestamp']
        #chequear si la lista initial_points_timestamp es diferente de vacio
        if len(initial_points_timestamp1) and len(final_points_timestamp1):
            vilaIsabel_departure_hours1 = bus_departure_hour(initial_points_timestamp1)
            first_point_departure_hour = bus_departure_hour (initial_points_timestamp1)
            last_point_departure_hour = bus_departure_hour (final_points_timestamp1)
            first_point_arrive_hour = bus_arrive_hour(initial_points_timestamp1)
            last_point_arrive_hour = bus_arrive_hour(final_points_timestamp1)
            my_time_table = create_travel_table(name_punto_inicio, first_point_arrive_hour, first_point_departure_hour, 'Vila Isabel', last_point_arrive_hour, last_point_departure_hour, busId, '438')
            new_df = new_df.append(my_time_table, ignore_index=True)
    return new_df

In [54]:
result = all_travel_table ('438', inverted_initial_point, 'Leblon', inverted_final_point, 'Vila Isabel', array_busId)
result

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,438,A27556,Vila Isabel_to_Leblon,2015-02-15 00:47:32,2015-02-15 01:44:32,0:57
1,438,A27556,Leblon_to_Vila Isabel,2015-02-15 01:44:32,2015-02-15 02:51:33,1:07
2,438,A27556,Vila Isabel_to_Leblon,2015-02-15 02:53:33,2015-02-15 03:49:33,0:56
3,438,A27556,Leblon_to_Vila Isabel,2015-02-15 03:49:33,2015-02-15 05:11:33,1:22
4,438,A27604,Vila Isabel_to_Leblon,2015-02-15 04:28:17,2015-02-15 10:12:17,5:44
5,438,A27604,Vila Isabel_to_Leblon,2015-02-15 06:43:17,2015-02-15 10:12:17,3:29
6,438,A27604,Vila Isabel_to_Leblon,2015-02-15 09:01:47,2015-02-15 10:12:17,1:10
7,438,A27604,Leblon_to_Vila Isabel,2015-02-15 10:12:17,2015-02-15 11:15:17,1:03
8,438,A27534,Vila Isabel_to_Leblon,2015-02-15 05:21:06,2015-02-15 12:08:33,6:47
9,438,A27534,Vila Isabel_to_Leblon,2015-02-15 08:04:05,2015-02-15 12:08:33,4:04
